# Marketing Mix Models 

## The Basics


Marketing Mix Modeling (MMM or MMX) is a statistical analysis technique used to evaluate the impact of a company's marketing activities on sales (or any variable that wants to be measured). It relies in analysing historical data and typically fitting a model in which sales acts as a dependent variable.

MMM helps businesses understand the effectiveness of different marketing channels (such as TV, online ads, promotions, and pricing) and how they interact with each other. This insight allows companies to allocate their marketing budgets more efficiently, optimize their marketing strategies. MMM is particularly useful because it quantifies the return on investment (ROI) of marketing expenditures, enabling data-driven decision-making.

It's a highly demanded skill that can be applied in virtually, any industry. 


## Objective of this Notebook 

This notebook serves as a guide on how to build MM models in order to obtain contributions of the different channels (touchpoints) on product sales. This toy example includes relevant feature transformations as adstock (decay), seasonality, saturation, lags, etc. 

Likewise, it has been built with toy data which mimics real-life scenarios, but is not a full picture of the marketing spends a real product has during a year.

The basic application of these type of models uses a traditional linear model, however, it does have a few limitations that will be explored across the Notebook.

## 1. Imports and setup

In [ ]:
# Imports.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

import optuna
# from scipy.interpolate import make_interp_spline
import plotly.express as px
import numpy as np
# import dabl


In [ ]:
# Set up 

# Configuration for viz and verbosity.
plt.rcParams["figure.figsize"] = (20, 8)
optuna.logging.set_verbosity(optuna.logging.WARNING)

# Path were the input file is stored.
data_path = "../data/cough_and_cold_sales.csv"



# 2. Helpers

In [ ]:
# Helpers
# Data process .

def get_median_filtered(signal: pd.Series, threshold: int = 3):
    
    """
    Función que nos permite sacar outliers utilizando el método de la mediana
    filtrada. Es bastante robusto a ruido, por lo que es ideal para series 
    temporales. 
     
    Directamente reemplaza outliers con la mediana!
    Info sobre el método en 
    https://en.wikipedia.org/wiki/Median_filter 
    :param signal: una serie o señal
    :param threshold: desviaciones respecto a la mediana    
    :return:   array igual a serie con los outliers reemplazados. 
    
    """
    
    outlier_check_signal = signal.copy()
    difference = np.abs(outlier_check_signal - np.median(outlier_check_signal))
    median_difference = np.median(difference)
    if median_difference == 0:
        s = 0
    else:
        s = difference / float(median_difference)
    mask = s > threshold
    outlier_check_signal[mask] = np.median(outlier_check_signal)
    return outlier_check_signal

def plot_outliers_signal(
        signal: pd.Series, threshold: int = 2, return_mask: bool = True
) -> None | np.ndarray:
    """_summary_

    Args:
        signal (pd.Series): _description_
        threshold (int, optional): _description_. Defaults to 3.
        return_mask (bool, optional): _description_. Defaults to True.

    Returns:
        None | np.ndarray: _description_
    """
    kw = dict(marker='o', linestyle='none', color='r', alpha=0.35)

    mds = get_median_filtered(signal, threshold=threshold)
    outlier_idx = np.where(mds != signal)[0]
    # make sure we use the series' index.
    outlier_idx = signal.index[outlier_idx]
    plt.figure(figsize=(10,8))

    plt.plot(signal, color = "darkblue")
    plt.plot(
        outlier_idx, signal[outlier_idx], **kw, 
        label = "Outliers"
    )
    plt.title("Outlier detection with cutoff {}".format(threshold))
    plt.legend()
    plt.show()
    if return_mask: 
      return outlier_idx


def _is_between(x, low, high) -> bool:
    if x >= low and x <= high:
        return 1
    else:
        return 0

def add_flag(
    df: pd.DataFrame,
    low: int,
    high: int,
):
    # add flag
    flag = df.apply(
        lambda row: 
            _is_between(
                row.name, low, high  # usig the index of the df.
        ),
        axis=1
    )
    return flag


# Carry over, Saturation helpers.

def carry_over_effect(
    input_variable: pd.Series, decay: float, lag: int
) -> pd.Series:
    "Apply carry over transformation to a given pandas Series"
    output = []
    for row in range(len(input_variable)):
        # Take care of observations with less days than required for lags. 
        # e.g. For the first day, we have a problem if lag=5.
        lag_aux = min(lag, row)
        
        # Apply decay effect to every row.
        output.append(
            sum(
                [
                    input_variable[row-i] * (decay ** i)  for i in range(lag_aux + 1)
                ]
            )
        )
    return pd.Series(output)

def saturation_effect(input_variable: pd.Series, alpha: float
) -> pd.Series:
    "Apply exponential transformation to a given pandas Series."
    return (1 - np.exp(-alpha * input_variable))

def adstock_tranformation(
    input_variable: pd.Series, alpha: float, decay: float, lag: int
) -> pd.Series:
    "Apply adstock transformation to a given pandas Series."
    return saturation_effect(
        carry_over_effect(
            input_variable, decay, lag
        ), alpha
    )

def add_adstock_to_pdf(
    pdf_input: pd.DataFrame,
    hyperparameters: dict,
    channels: list,
) -> pd.DataFrame:
    "Add adstock columns to a given dataset."
    
    pdf = pdf_input.copy()
    pdf.reset_index(drop=True, inplace=True)
    # Add a column for each channel with the name adstock_channel.
    for c in channels:
        pdf[f'adstock_{c}'] = adstock_tranformation(
            pdf[c],
            hyperparameters[f'alpha_{c}'],
            hyperparameters[f'decay_{c}'],
            hyperparameters[f'lag_{c}']
        ).values
    return pdf

def sales_from(
    channel: str,
    features: list,
    model: LinearRegression,
    X: pd.DataFrame
) -> pd.DataFrame:
    "Compute sales attributed to a certain channel"
    coef = model.coef_[features.index(channel)]
    obs = X.iloc[:, features.index(channel)]
    return  coef * obs

def create_stack_df(_df, features: list, lr: LinearRegression):
    plot_df = pd.DataFrame()

    for f in _df.columns:
        if "adstock" in f:
            _series = sales_from(f, features, lr, _df)
        elif "index" in f:
            _series = sales_from(f, features, lr, _df)

        _series.name = f
        plot_df = pd.concat([plot_df, _series], axis=1)
    return plot_df



def performance_metrics(true: np.ndarray, pred: np.ndarray) -> None:

    r2 = r2_score(
        true,
        pred
    )

    mape = mean_squared_error(
        true,
        pred
    )

    print("-------------------")
    print()
    print(f'MSE:{np.round(mape, 3)}')
    print(f'R2: {np.round(r2, 3)}')
    print()

## 3. EDA and basic transformations

Here we have synthetic weekly data of the sales of an off the counter **(OTC) cough and cold medicine** in a particular country. Along with the sales data, we have marketing spends for different channels that are typically relevant in the pharma industry. In this simple toy example we have a normalized monetary spend for tv, social media, medical crongresses, and trade (expenditure related to in-pharmacy display). We also have two external variables, the `stringency_index` which is related to the severness of COVID restrictions in this country, and the `flu_index` which is a variable monitored by the World Health Organization to track the intensity of the flu season in a particular region.  

Our objective is to determine the ROI of each of our mtk channels, to enable data-drive decision making for allocating our future marketing budget. 

In [ ]:
# Define channels to use.
CHANNELS: list[str] = ['tv', 'social_media', 'congress', 'trade']
EXT_VARS: list[str] = ['flu_index', 'stringency_index'] 
# Define target variable.
TARGET = 'sales'
# Define weeks for testing period.
TEST_SIZE = 8
# Seed for reproducibility
SEED = 123456

In [ ]:
# Load data
df = pd.read_csv(data_path, sep= ";", parse_dates=["date"])
df.head()

In [ ]:
# set date as index as it will make plotting easier
df.set_index("date", inplace=True)

In [ ]:
# Plot the target var and its Outliers
outlier_index = plot_outliers_signal(
    df[TARGET], threshold=2
)
plt.show()



We have an evident abnormal event that happened in 2020 and 2021 that collapsed the sales. There's also a high spike in March 2020. This is of course, due to the pandemic, where there was a high "panic buying" spike, and later, a heavy drop in sales. 

What to do here?

Many people decide to leave out the pandemic years. However, the effects of COVID could still be felt up until early 2022, which leaves us with very little data for fitting our model. What's best depends on the use case, however, for this example, let's add a flag that resembles the effects of the pandemic.

In [ ]:
# Adding a Covid flag
df['covid_flag'] = add_flag(
    df,
    outlier_index.min(),
    outlier_index.max()
)

In [ ]:
df.head(2) 

## Interactive plots

In [ ]:
# Plot it
px.line(df.sales)

In [ ]:
# PLot channel spend for month
fig = px.bar(
    df, 
    x=df.index, 
    y=CHANNELS, 
    title="Channel Spend per Date"
)
fig.show()

## 4. Adstock modeling.

In this section we add (i) carry over and (ii) saturation effects.

We want to model the relationship of the different investment channels using a linear regression, however we want to take into account the [decay effect](https://en.wikipedia.org/wiki/Advertising_adstock#Advertising_lag:_decay_effect) and the [law of diminishing returns](https://en.wikipedia.org/wiki/Advertising_adstock#Campaign_carry-over).

---
For this, instead of working directly with spend per channel, we apply different transformations to them

> 1. For carry over $c_{t}$, we apply the following transformation.
$$c_{t} = x_{t} + \sum_{j=1}^{n} \lambda^{t}  x_{t-j}$$
>
> Where $x_{t}$ is the investment in the channel for the $t$ period, $n$ represents the amount of periods to look back and $\lambda$ represents the strength of the decay factor. This means that investments made in time $t$ will have still an impact of the following weeks, and this effect will decay over time. Both $n$ and $\lambda$ will be parameters of the function.
><br/>
><br/>
>2. For the saturation effect $s_{t}$, the following transformation is applied.
>
>$$s_{t} =1 -  e^{-\alpha x_{t}} $$
>
> In this case, $\alpha$ will be a parameter to input.

<br/>

In consequence, instead of modeling sales as a function of investment per channel, we fit a regression to the transformations described previously. The question now is which values should we use for **$n$, $\lambda$ and $\alpha$**? We can think of them as *hyperparameters of our model*. And then perform a *numerical optimization* to find values that minimize the mean squared error (or any metric of our interest) of the model.
<br/>


Lastly, data presents a heavy seasonality, one hot encoded variables were added for every month in the year.

---

For a complete explanation of the methods used in here, we refer the reader to page *154/510* of [Introduction to Algorithmic Marketing](https://algorithmicweb.files.wordpress.com/2018/07/algorithmic-marketing-ai-for-marketing-operations-r1-7g.pdf).


## 5. Fit model.

### Tune hyperparameters

- We use optuna for hyperparameter tuning, but other libraries like `scipy.minimize` should work as well.
- For every channel, we have three ($\alpha$, $n$ and $\lambda$) hyperparameters to tune.
- We work with a traditional linear regression. -> prone to overfitting!!
- We aim to minimize *MSE*.
- We repeat 10000 trials for the optimization, caution! can take some time. Feel free to change `n_trials=100` to speed things up.
- We split the dataset in train and test. Since we are working with time series we work with consecutives chunks of data.

In [ ]:
# Split data for train test.
df_train = df.iloc[: -TEST_SIZE, :]
df_test = df.iloc[-TEST_SIZE:, :]

# Define features and target variable.
features = [
    c for c in list(df.columns) if 'flag' in c or 'index' in c
] + [f'adstock_{c}' for c in CHANNELS]



In [ ]:
# Usually helpers should be at the top of the notebook, but we put this one here
# as it is a dictionary with multiple values to configure and play around with.

def objective_tune_params(trial, _df: pd.DataFrame = df_train):
    
    # we need to optimize and learn alpha, decay and lag
    # on the suggestions, if we have expert or prior knowledge we can inform this better
    # however, for this exercise, we leave these defaults.
    hyperparameters: dict = {
        'alpha_tv': trial.suggest_float('alpha_tv', 0, 0.1),
        'decay_tv': trial.suggest_float('decay_tv',  0, 0.5),
        'lag_tv': trial.suggest_int('lag_tv', 0, 6),
        'alpha_social_media': trial.suggest_float('alpha_social_media', 0, 0.1),
        'decay_social_media': trial.suggest_float('decay_social_media',  0, 0.5),        
        'lag_social_media': trial.suggest_int('lag_social_media', 0, 6),
        'alpha_congress': trial.suggest_float('alpha_congress', 0, 0.1),
        'decay_congress': trial.suggest_float('decay_congress',  0, 0.5),
        'lag_congress': trial.suggest_int('lag_congress', 0, 6),
        'alpha_trade': trial.suggest_float('alpha_trade', 0, 0.1),
        'decay_trade': trial.suggest_float('decay_trade',  0, 0.5),
        'lag_trade': trial.suggest_int('lag_trade', 0, 6)
    }
    
    # Add adstock variables
    df_adstock = add_adstock_to_pdf(_df, hyperparameters, CHANNELS)

    # Define X and y.  
    X = df_adstock[features]
    y = df_adstock[TARGET]
    
    # Initialize regression class.
    #lr = Ridge()
    lr = LinearRegression()
    
    # Fit model and compute error (mean squared error).
    lr.fit(X, y)
    y_hat = lr.predict(X)
    return np.mean((y - y_hat) ** 2)

In [ ]:
# Perform the optimization.
study_tune_params = optuna.create_study(
    sampler=optuna.samplers.RandomSampler(seed=SEED)
)
study_tune_params.optimize(objective_tune_params, n_trials=1000)

### Run final model.
With tuned hyperparameters, we train the final model.

In [ ]:
# Check optimized params.
study_tune_params.best_params

In [ ]:
# Best set of hyperparameters.
hyperparameters = study_tune_params.best_params

# We replicate adstock transformation, train test splitting and model fitting. 
# Add adstock columns. 
df_adstock_train  = add_adstock_to_pdf(df_train, hyperparameters, CHANNELS) 
df_adstock_test = add_adstock_to_pdf(df_test, hyperparameters, CHANNELS)

# Define train and test data.
X_train, y_train = df_adstock_train[features], df_adstock_train[TARGET]
X_test, y_test = df_adstock_test[features], df_adstock_test[TARGET]

# Initialize model.
lr = LinearRegression()

lr.fit(X_train, y_train)

## 6. Model performance.

In [ ]:
# Line chart for performance.

y_train.index = df_train.index  # adding index back for convenience

# Plot it
actual_vs_pred = pd.DataFrame(
    {
        "actual": y_train,
        "pred": lr.predict(X_train),
       
    },
    index=df_train.index
)
actual_vs_pred.plot(title="Actual vs prediction")

plt.show()


print("Training Performance")
performance_metrics(y_train, lr.predict(X_train))

print("Test Performance")
performance_metrics(y_test, lr.predict(X_test))



## 7. Channel contribution.

We observe the marginal channel contribution and the return on investment per channel.



In [ ]:

# base sales takes into account the intercept plus monthly effects.
base = (
    sum(
        [
            lr.coef_[
                features.index(c)
            ] * X_train.iloc[:, features.index(c)]
            for c in features if 'adstock' not in c
        ]
    )
      + [lr.intercept_] * len(df_train.index))

# Computation of contribution S = B + b1*C1 + b2*C2 
# Cnt_2 = (B+B1*C1+B2*C2) - (B+B1*C1) = B2*C2 


In [ ]:
# create contribution plot
plot_df = create_stack_df(X_train, features, lr)
# re adding base. 
base.name = "base"
plot_df = pd.concat([base, plot_df], axis=1)

In [ ]:
px.area(
    plot_df,
    title="Contribution Painting"
)

In [ ]:

# Compute ROI for every channel.
for c in features:
    if 'index' not in c and 'flag' not in c:
        channel_share = np.round(sum(sales_from(c, features, lr, X_train)) / sum(lr.predict(X_train)), 2)
        channel_roi = np.round(sum(sales_from(c, features, lr, X_train)) / sum(df_adstock_train[c.replace('adstock_', '')]), 2)
        print(f'For {c}: share of total sales is {channel_share} and ROI is {channel_roi}')

In [ ]:
px.bar(
        y=lr.coef_,
        x=X_train.columns,
        title='Contribution bar plot',
        labels={
            "x": "Feature",
            "y": "Contribution"
        }
)


# Discussion 

- Why are some channel contributions negative? Does this make sense from a business POV? 
- What about scaling the data?
- What about seasonality?
- What would happen if we try Log or Semi-log models? Would it be worth it?
- Would it be worth it to use another type of model, for example tree-based algorithms?
- What recommendations would you give to business with these results